## 1. Import Required Libraries

In [32]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from torch.utils.data import random_split
from torchaudio.datasets import LJSPEECH

# Import from our modules
from models import MLP, CNN, RNN_LSTM, RNN_GRU, RNN_BiLSTM
from data import TextEncoder, FeatureExtractor, create_dataloaders, greedy_decode
from trainer import TrainingConfig, Trainer, CNNTrainer, ModelComparator

print("✓ Libraries imported successfully")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

✓ Libraries imported successfully
PyTorch version: 2.9.1+cu128
CUDA available: False


## 2. Setup Device and Load Dataset

In [33]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# crate dir for data if not exists
import os
if not os.path.exists('./data'):
    os.makedirs('./data')

# Load dataset
print("Loading LJSPEECH dataset...")
dataset = LJSPEECH(root="./data", download=True)
print(f"Total samples: {len(dataset)}")

# Split into train/test
train_len = int(len(dataset) * 0.9)
test_len = len(dataset) - train_len
train_dataset, test_dataset = random_split(dataset, [train_len, test_len])
print(f"Train set: {train_len} | Test set: {test_len}")

Device: cpu
Loading LJSPEECH dataset...
Total samples: 13100
Train set: 11790 | Test set: 1310


## 3. Initialize Text Encoder and Feature Extractors

In [34]:
# Text encoder
text_encoder = TextEncoder()
print(f"Number of classes: {text_encoder.num_classes}")
print(f"Character set: {text_encoder.symbols}")

# Feature extractors
feature_extractor_mfcc = FeatureExtractor(feature_type='mfcc', sample_rate=22050)
feature_extractor_mel = FeatureExtractor(feature_type='mel', sample_rate=22050)

print("✓ Text encoder and feature extractors ready")

Number of classes: 39
Character set: _-!'(),.:;? abcdefghijklmnopqrstuvwxyz
✓ Text encoder and feature extractors ready


## 4. Create Data Loaders

In [35]:
# Create dataloaders for MFCC
print("Creating MFCC dataloaders...")
train_loader_mfcc, test_loader_mfcc = create_dataloaders(
    train_dataset, test_dataset,
    feature_extractor_mfcc, text_encoder,
    batch_size=16, num_workers=0
)
print(f"✓ MFCC loaders created ({len(train_loader_mfcc)} train batches)")

# Create dataloaders for MelSpectrogram
print("Creating MelSpectrogram dataloaders...")
train_loader_mel, test_loader_mel = create_dataloaders(
    train_dataset, test_dataset,
    feature_extractor_mel, text_encoder,
    batch_size=16, num_workers=0
)
print(f"✓ MelSpectrogram loaders created ({len(train_loader_mel)} train batches)")

Creating MFCC dataloaders...
✓ MFCC loaders created (737 train batches)
Creating MelSpectrogram dataloaders...
✓ MelSpectrogram loaders created (737 train batches)


## 5. Define Decoding Function

In [36]:
def decode_fn(log_probs):
    """Wrapper for greedy decoding."""
    return greedy_decode(log_probs, text_encoder)

print("✓ Decoding function ready")

✓ Decoding function ready


## 6. Train MLP + MFCC Model

### Note: Training takes time
The models will train for 2 epochs on the LJSPEECH dataset (~13,000 samples). This may take 30+ minutes depending on your hardware. For faster testing, you can:
1. Reduce `num_epochs` to 1
2. Use a smaller batch size
3. Limit the dataset size

In [ ]:
print("\n" + "=" * 70)
print("TRAINING: MLP + MFCC")
print("=" * 70)

config = TrainingConfig(
    num_epochs=2,
    learning_rate=1e-3,
    batch_size=16,
    grad_clip=1.0,
    device=device
)

# MLP with MFCC (128 features)
model_mlp = MLP(
    in_dim=128,
    hidden=256,
    num_classes=text_encoder.num_classes
)

trainer_mlp = Trainer(model_mlp, config, blank_idx=0)
metrics_mlp = trainer_mlp.train(
    train_loader_mfcc,
    test_loader_mfcc,
    decode_fn,
    "MLP + MFCC"
)
trainer_mlp.save("mlp_ctc.pth")

print("✓ MLP model training complete")


TRAINING: MLP + MFCC
Training: MLP + MFCC
Parameters: 43,047
Device: cpu
  Epoch 1 Batch 50: loss = 12.3322
  Epoch 1 Batch 50: loss = 12.3322
  Epoch 1 Batch 100: loss = 5.0888
  Epoch 1 Batch 100: loss = 5.0888
  Epoch 1 Batch 150: loss = 3.2678
  Epoch 1 Batch 150: loss = 3.2678
  Epoch 1 Batch 200: loss = 3.0670
  Epoch 1 Batch 200: loss = 3.0670
  Epoch 1 Batch 250: loss = 3.0619
  Epoch 1 Batch 250: loss = 3.0619
  Epoch 1 Batch 300: loss = 3.0069
  Epoch 1 Batch 300: loss = 3.0069
  Epoch 1 Batch 350: loss = 3.0596
  Epoch 1 Batch 350: loss = 3.0596


## 7. Train CNN + MelSpectrogram Model

In [ ]:
print("\n" + "=" * 70)
print("TRAINING: CNN + MelSpectrogram")
print("=" * 70)

model_cnn = CNN(
    n_mels=64,
    num_classes=text_encoder.num_classes
)

trainer_cnn = CNNTrainer(model_cnn, config, blank_idx=0)
metrics_cnn = trainer_cnn.train(
    train_loader_mel,
    test_loader_mel,
    decode_fn,
    "CNN + MelSpectrogram"
)
trainer_cnn.save("cnn_mel_ctc.pth")

print("✓ CNN model training complete")

## 8. Train LSTM + MelSpectrogram Model

In [ ]:
print("\n" + "=" * 70)
print("TRAINING: LSTM + MelSpectrogram")
print("=" * 70)

model_lstm = RNN_LSTM(
    input_dim=64,
    hidden_dim=256,
    num_layers=2,
    num_classes=text_encoder.num_classes
)

trainer_lstm = Trainer(model_lstm, config, blank_idx=0)
metrics_lstm = trainer_lstm.train(
    train_loader_mel,
    test_loader_mel,
    decode_fn,
    "LSTM + MelSpectrogram"
)
trainer_lstm.save("lstm_ctc.pth")

print("✓ LSTM model training complete")

## 9. Train GRU + MelSpectrogram Model

In [ ]:
print("\n" + "=" * 70)
print("TRAINING: GRU + MelSpectrogram")
print("=" * 70)

model_gru = RNN_GRU(
    input_dim=64,
    hidden_dim=256,
    num_layers=2,
    num_classes=text_encoder.num_classes
)

trainer_gru = Trainer(model_gru, config, blank_idx=0)
metrics_gru = trainer_gru.train(
    train_loader_mel,
    test_loader_mel,
    decode_fn,
    "GRU + MelSpectrogram"
)
trainer_gru.save("gru_ctc.pth")

print("✓ GRU model training complete")

## 10. Train BiLSTM + MelSpectrogram Model

In [ ]:
print("\n" + "=" * 70)
print("TRAINING: BiLSTM + MelSpectrogram")
print("=" * 70)

model_bilstm = RNN_BiLSTM(
    input_dim=64,
    hidden_dim=256,
    num_layers=2,
    num_classes=text_encoder.num_classes
)

trainer_bilstm = Trainer(model_bilstm, config, blank_idx=0)
metrics_bilstm = trainer_bilstm.train(
    train_loader_mel,
    test_loader_mel,
    decode_fn,
    "BiLSTM + MelSpectrogram"
)
trainer_bilstm.save("bilstm_ctc.pth")

print("✓ BiLSTM model training complete")

## 11. Compare Model Performance

In [ ]:
# Initialize comparator
comparator = ModelComparator()

# Add all results
comparator.add_result("MLP + MFCC", metrics_mlp)
comparator.add_result("CNN + MelSpectrogram", metrics_cnn)
comparator.add_result("LSTM + MelSpectrogram", metrics_lstm)
comparator.add_result("GRU + MelSpectrogram", metrics_gru)
comparator.add_result("BiLSTM + MelSpectrogram", metrics_bilstm)

# Print comparison
comparator.print_comparison()

## 12. Visualize Results

In [ ]:
# Create comparison visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Model Performance Comparison', fontsize=16, fontweight='bold')

# 1. Loss Convergence
ax = axes[0, 0]
for model_name in ["MLP + MFCC", "CNN + MelSpectrogram", "LSTM + MelSpectrogram", 
                    "GRU + MelSpectrogram", "BiLSTM + MelSpectrogram"]:
    metrics = comparator.results[model_name]
    ax.plot(metrics['epochs'], metrics['train_losses'], marker='o', label=model_name, linewidth=2)
ax.set_xlabel('Epoch', fontsize=11)
ax.set_ylabel('Training Loss', fontsize=11)
ax.set_title('Loss Convergence', fontsize=12, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# 2. Training Time per Epoch
ax = axes[0, 1]
models = ["MLP", "CNN", "LSTM", "GRU", "BiLSTM"]
times_per_epoch = []
for model_name in ["MLP + MFCC", "CNN + MelSpectrogram", "LSTM + MelSpectrogram", 
                    "GRU + MelSpectrogram", "BiLSTM + MelSpectrogram"]:
    metrics = comparator.results[model_name]
    avg_time = sum(metrics['train_times']) / len(metrics['train_times']) if metrics['train_times'] else 0
    times_per_epoch.append(avg_time)

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
bars = ax.bar(models, times_per_epoch, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.set_ylabel('Time per Epoch (seconds)', fontsize=11)
ax.set_title('Training Speed', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, t in zip(bars, times_per_epoch):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{t:.1f}s', ha='center', va='bottom', fontweight='bold', fontsize=9)

# 3. Parameter Count
ax = axes[1, 0]
params_sorted = sorted([
    ("MLP", comparator.results["MLP + MFCC"]['param_count']),
    ("CNN", comparator.results["CNN + MelSpectrogram"]['param_count']),
    ("LSTM", comparator.results["LSTM + MelSpectrogram"]['param_count']),
    ("GRU", comparator.results["GRU + MelSpectrogram"]['param_count']),
    ("BiLSTM", comparator.results["BiLSTM + MelSpectrogram"]['param_count']),
], key=lambda x: x[1])

names, params = zip(*params_sorted)
bars = ax.barh(names, params, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.set_xlabel('Number of Parameters', fontsize=11)
ax.set_title('Model Size', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

# Add value labels
for bar, p in zip(bars, params):
    width = bar.get_width()
    ax.text(width, bar.get_y() + bar.get_height()/2.,
            f'{p:,.0f}', ha='left', va='center', fontweight='bold', fontsize=9, 
            bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.3))

# 4. Summary Statistics Table
ax = axes[1, 1]
ax.axis('tight')
ax.axis('off')

# Create table data
table_data = []
for model_name in ["MLP + MFCC", "CNN + MelSpectrogram", "LSTM + MelSpectrogram", 
                    "GRU + MelSpectrogram", "BiLSTM + MelSpectrogram"]:
    metrics = comparator.results[model_name]
    total_time = sum(metrics['train_times'])
    final_loss = metrics['train_losses'][-1] if metrics['train_losses'] else 0
    model_short = model_name.split()[0]
    table_data.append([
        model_short,
        f"{metrics['param_count']:,}",
        f"{total_time:.1f}s",
        f"{final_loss:.4f}"
    ])

table = ax.table(
    cellText=table_data,
    colLabels=['Model', 'Parameters', 'Time (2 epochs)', 'Final Loss'],
    cellLoc='center',
    loc='center',
    colWidths=[0.15, 0.25, 0.25, 0.25]
)
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2)

# Style header
for i in range(4):
    table[(0, i)].set_facecolor('#4CAF50')
    table[(0, i)].set_text_props(weight='bold', color='white')

# Style rows
for i in range(1, 6):
    for j in range(4):
        if i % 2 == 0:
            table[(i, j)].set_facecolor('#f0f0f0')
        else:
            table[(i, j)].set_facecolor('#ffffff')

ax.set_title('Performance Summary', fontsize=12, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Comparison visualizations saved to 'model_comparison.png'")

## 13. Detailed Analysis and Recommendations

In [ ]:
print("\n" + "=" * 80)
print("DETAILED MODEL ANALYSIS")
print("=" * 80)

print("""
╔════════════════════════════════════════════════════════════════════════╗
║                        MODEL CHARACTERISTICS                            ║
╚════════════════════════════════════════════════════════════════════════╝

📊 MLP + MFCC
   ├─ Architecture: Simple feedforward network
   ├─ Temporal Modeling: None (independent timesteps)
   ├─ Best For: Fast baseline, testing pipeline
   ├─ Pros:
   │  ✓ Very fast training
   │  ✓ Minimal memory usage
   │  ✓ Easy to understand
   └─ Cons:
      ✗ Poor accuracy
      ✗ No temporal context
      ✗ Not suitable for real STT

🖥️  CNN + MelSpectrogram
   ├─ Architecture: 2 conv layers + FC layers
   ├─ Temporal Modeling: Local (convolutional kernels)
   ├─ Best For: Fast feature extraction
   ├─ Pros:
   │  ✓ Faster than RNNs
   │  ✓ Good local feature learning
   │  ✓ Moderate memory usage
   └─ Cons:
      ✗ Limited long-range dependencies
      ✗ Fixed receptive field
      ✗ Medium accuracy

🔄 LSTM + MelSpectrogram
   ├─ Architecture: 2-layer LSTM
   ├─ Temporal Modeling: Long-range (forward direction)
   ├─ Best For: Balanced accuracy/speed
   ├─ Pros:
   │  ✓ Handles long sequences
   │  ✓ Stable gradient flow
   │  ✓ Industry standard
   └─ Cons:
      ✗ Slower than GRU/CNN
      ✗ More parameters than GRU
      ✗ Cannot see future context

⚡ GRU + MelSpectrogram
   ├─ Architecture: 2-layer GRU
   ├─ Temporal Modeling: Long-range (forward direction)
   ├─ Best For: Fast RNN training
   ├─ Pros:
   │  ✓ Faster than LSTM (~25%)
   │  ✓ Fewer parameters
   │  ✓ Similar accuracy to LSTM
   └─ Cons:
      ✗ Slightly less expressive
      ✗ Cannot see future context
      ✗ Rare in modern STT

⭐ BiLSTM + MelSpectrogram
   ├─ Architecture: 2-layer Bidirectional LSTM
   ├─ Temporal Modeling: Long-range (both directions)
   ├─ Best For: Maximum accuracy (offline STT)
   ├─ Pros:
   │  ✓ BEST accuracy for offline tasks
   │  ✓ Sees past AND future context
   │  ✓ Longest effective receptive field
   └─ Cons:
      ✗ 2x slower than LSTM
      ✗ 2x memory than LSTM
      ✗ Cannot be used for streaming
""")

print("\n" + "=" * 80)
print("RECOMMENDATIONS FOR YOUR USE CASE")
print("=" * 80)
print("""
🎯 CHOOSE BiLSTM IF:
   • You need maximum accuracy
   • Processing is offline (not real-time)
   • Computational resources are available
   → BEST for: Batch speech transcription, research

🎯 CHOOSE GRU IF:
   • You want LSTM-like performance but faster
   • You have limited GPU memory
   • You need quick prototyping
   → BEST for: Real-time systems with constraints

🎯 CHOOSE LSTM IF:
   • You want industry-standard stability
   • You need long-range dependency modeling
   • You have balanced resources
   → BEST for: Production deployments

🎯 CHOOSE CNN IF:
   • You need very fast inference
   • Latency is critical (streaming)
   • You have very limited resources
   → BEST for: Edge devices, real-time streaming

🎯 CHOOSE MLP ONLY FOR:
   • Testing the data pipeline
   • Understanding the problem
   • Benchmarking
   → AVOID for: Actual STT tasks
""")

print("\n" + "=" * 80)
print("TRAINING EFFICIENCY METRICS")
print("=" * 80)

# Calculate efficiency
for model_name in ["MLP + MFCC", "CNN + MelSpectrogram", "LSTM + MelSpectrogram", 
                    "GRU + MelSpectrogram", "BiLSTM + MelSpectrogram"]:
    metrics = comparator.results[model_name]
    total_time = sum(metrics['train_times'])
    final_loss = metrics['train_losses'][-1] if metrics['train_losses'] else 0
    params = metrics['param_count']
    
    # Efficiency metrics
    loss_per_second = final_loss / total_time if total_time > 0 else 0
    params_per_second = params / total_time if total_time > 0 else 0
    
    print(f"\n{model_name}:")
    print(f"  Total Training Time:  {total_time:.1f}s")
    print(f"  Final Loss:           {final_loss:.4f}")
    print(f"  Parameters:           {params:,}")
    print(f"  Loss Reduction/sec:   {loss_per_second:.6f}")

## 14. Save Results and Summary

In [ ]:
# Create summary dataframe
summary_data = []

for model_name in ["MLP + MFCC", "CNN + MelSpectrogram", "LSTM + MelSpectrogram", 
                    "GRU + MelSpectrogram", "BiLSTM + MelSpectrogram"]:
    metrics = comparator.results[model_name]
    total_time = sum(metrics['train_times'])
    initial_loss = metrics['train_losses'][0] if metrics['train_losses'] else 0
    final_loss = metrics['train_losses'][-1] if metrics['train_losses'] else 0
    improvement = ((initial_loss - final_loss) / initial_loss * 100) if initial_loss > 0 else 0
    
    summary_data.append({
        'Model': model_name,
        'Parameters': metrics['param_count'],
        'Total Time (s)': round(total_time, 1),
        'Avg Time/Epoch (s)': round(total_time / len(metrics['train_times']), 1),
        'Initial Loss': round(initial_loss, 4),
        'Final Loss': round(final_loss, 4),
        'Improvement (%)': round(improvement, 1)
    })

summary_df = pd.DataFrame(summary_data)
print("\n" + "=" * 80)
print("COMPLETE SUMMARY TABLE")
print("=" * 80)
print(summary_df.to_string(index=False))

# Save to CSV
summary_df.to_csv('training_summary.csv', index=False)
print("\n✓ Summary saved to 'training_summary.csv'")

# Save metrics as JSON
import json
metrics_dict = {}
for model_name, metrics in comparator.results.items():
    metrics_dict[model_name] = {
        'param_count': int(metrics['param_count']),
        'epochs': metrics['epochs'],
        'train_losses': [float(x) for x in metrics['train_losses']],
        'train_times': [float(x) for x in metrics['train_times']]
    }

with open('metrics.json', 'w') as f:
    json.dump(metrics_dict, f, indent=2)
print("✓ Metrics saved to 'metrics.json'")

print("\n" + "=" * 80)
print("TRAINING AND EVALUATION COMPLETE!")
print("=" * 80)
print("\nGenerated files:")
print("  • model_comparison.png - Visual comparison of all models")
print("  • training_summary.csv - Summary metrics table")
print("  • metrics.json - Detailed metrics for further analysis")
print("  • *_ctc.pth - Trained model weights")